# Data Preprocessing 

In [2]:
import pandas as pd 
import numpy as np
from os import path

## Loading Data

In [3]:
year = ['2020', '2021']
patient = []
symptom = []
vaccine = []
for y in year:
    folder = '{}VAERSData'.format(y)
    df = pd.read_csv(path.join(folder, '{}VAERSDATA.csv'.format(y)), encoding='latin1')
    patient.append(df)
    df = pd.read_csv(path.join(folder, '{}VAERSSYMPTOMS.csv'.format(y)), encoding='latin1')
    symptom.append(df)
    df = pd.read_csv(path.join(folder, '{}VAERSVAX.csv'.format(y)), encoding='latin1')
    vaccine.append(df)

/Users/jlee/covid-adv/covid-venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7,9,10,12,15,16,23,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/jlee/covid-adv/covid-venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,916600,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,916601,COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
2,916602,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,916603,COVID19,MODERNA,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
4,916604,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
...,...,...,...,...,...,...,...,...
411745,1427468,COVID19,PFIZER\BIONTECH,EW0167,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
411746,1427471,COVID19,MODERNA,011D21A,1,IM,LA,COVID19 (COVID19 (MODERNA))
411747,1427472,COVID19,MODERNA,033b21a,2,IM,LA,COVID19 (COVID19 (MODERNA))
411748,1427475,COVID19,MODERNA,006M20A,2,NaN,NaN,COVID19 (COVID19 (MODERNA))


## Data Cleaning

### Filtering out non-COVID19 vaccinated patients' records

In [17]:
# Unique vaccine types
print(vaccine[0].VAX_TYPE.unique())

# Filtering by vaccine type
for i in range(len(vaccine)):
    vaccine[i]= vaccine[i].loc[vaccine[i]['VAX_TYPE']=='COVID19']
    
# Filtering by VAERS_ID 
for i in range(len(vaccine)):
    v_id = list(vaccine[i]['VAERS_ID'].unique()) 
    patient[i] = patient[i][patient[i]['VAERS_ID'].isin(v_id)]
    symptom[i] = symptom[i][symptom[i]['VAERS_ID'].isin(v_id)]    
    
# Combine two dataframes
pat = pd.concat([patient[0], patient[1]])
sym = pd.concat([symptom[0], symptom[1]])
vac = pd.concat([vaccine[0], vaccine[1]])

['VARZOS' 'UNK' 'FLU3' 'FLUR4' 'FLU4' 'PPV' 'PNC13' 'DTAPIPV' 'HEP'
 'FLUC4' 'HEPA' 'HPV9' 'FLUX' 'MNQ' 'DTAP' 'MMR' 'MMRV' 'MENB' 'VARCEL'
 'RV1' 'RV5' 'TDAP' 'FLUN4' 'FLUA3' 'MEN' 'IPV' 'DTAPHEPBIP' 'HEPAB'
 'DTAPIPVHIB' 'HPV4' 'HIBV' 'TYP' 'DTP' 'RAB' 'TD' 'PNC10' 'FLUN3' 'ANTH'
 'YF' 'JEV1' 'ADEN_4_7' 'SMALL' 'HPVX' 'PNC' 'DT' 'TTOX' 'CHOL' 'FLUR3'
 'RVX' 'HPV2' 'BCG' 'JEVX' 'DTOX' 'HBHEPB' 'FLUA4' 'RUB' 'FLUC3' 'MENHIB'
 'COVID19' 'DF' 'FLUX(H1N1)' 'EBZR']


In [ ]:
sym.to_csv('covid_adv_sym.csv', index = False)

### Patient Data

In [18]:
# Delete irrelevant columns from patient data
irr_columns = ['RECVDATE', 'STATE', 'CAGE_YR', 'CAGE_MO', 'RPT_DATE', 'LAB_DATA',
             'V_ADMINBY', 'V_FUNDBY', 'SPLTTYPE', 'FORM_VERS','TODAYS_DATE', 
             'ER_VISIT', 'OFC_VISIT', 'ER_ED_VISIT']
pat.drop(columns = irr_columns, inplace = True)

pat.columns 

Index(['VAERS_ID', 'AGE_YRS', 'SEX', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED',
       'L_THREAT', 'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD',
       'VAX_DATE', 'ONSET_DATE', 'NUMDAYS', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY',
       'PRIOR_VAX', 'BIRTH_DEFECT', 'ALLERGIES'],
      dtype='object')

In [19]:
# Replace NaNs to 'No'
cols = ['DIED', 'DISABLE', 'L_THREAT', 'HOSPITAL','BIRTH_DEFECT']
for c in cols:
    pat[c] = pat[c].fillna('No')
    pat[c] = pat[c].replace('Y', 'Yes')
    
# Format answers 
pat['RECOVD'] = pat['RECOVD'].fillna('Unknown')
pat['RECOVD'] = pat['RECOVD'].replace('Y', 'Yes')
pat['RECOVD'] = pat['RECOVD'].replace('N', 'No')
pat['RECOVD'] = pat['RECOVD'].replace('U', 'Unknown')

### Vaccine Data

In [20]:
# Finding all duplicate rows based on VAERS_ID
dups = vac[vac.duplicated(['VAERS_ID'])]
vac_info = vac.drop_duplicates(subset = ['VAERS_ID'], keep = 'last')

In [21]:
vac_info = vac_info[['VAERS_ID', 'VAX_MANU', 'VAX_DOSE_SERIES']]

In [22]:
# Adding vaccine info
pat = pd.merge(pat, vac_info, how = 'left', on = ['VAERS_ID'])

In [23]:
pat

,VAERS_ID,AGE_YRS,SEX,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,HOSPITAL,HOSPDAYS,X_STAY,...,ONSET_DATE,NUMDAYS,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,BIRTH_DEFECT,ALLERGIES,VAX_MANU,VAX_DOSE_SERIES
0,1410490,54.0,F,Fever Narrative:,No,NaN,No,No,NaN,NaN,...,01/13/2021,0.0,NaN,NaN,NaN,NaN,No,NaN,PFIZER\BIONTECH,2
1,1413866,67.0,M,"paresthesia, lightheadedness. Narrative: At ...",No,NaN,No,No,NaN,NaN,...,03/18/2021,0.0,NaN,NaN,NaN,NaN,No,NaN,MODERNA,1
2,896636,47.0,F,"ARM SWELLING WITH PAIN, FEVER, FATIGUE, THEN ...",No,NaN,No,No,NaN,NaN,...,10/02/2020,4.0,10mg Prozac daily,none,none,NaN,No,tetracycline,MODERNA,UNK
3,902418,56.0,F,Patient experienced mild numbness traveling fr...,No,NaN,No,No,NaN,NaN,...,12/15/2020,0.0,latex,none,none,NaN,No,none,PFIZER\BIONTECH,1
4,902440,35.0,F,C/O Headache,No,NaN,No,No,NaN,NaN,...,12/15/2020,0.0,NaN,NaN,NaN,NaN,No,NaN,PFIZER\BIONTECH,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404844,1427468,17.0,F,"Prevaccination, patient reported history of an...",No,NaN,No,No,NaN,NaN,...,06/22/2021,0.0,Unknown,Unknown,Unknown,Fainting,No,"""Cillians""",PFIZER\BIONTECH,1
404845,1427471,18.0,M,"Systemic: Dizziness / Lightheadness-Mild, Syst...",No,NaN,No,No,NaN,NaN,...,06/24/2021,0.0,NaN,NaN,NaN,NaN,No,NaN,MODERNA,1
404846,1427472,54.0,F,After about 5 minutes of administration of the...,No,NaN,No,No,NaN,NaN,...,06/25/2021,0.0,10 ml of Benadryl Liquid 12.5mg/5ml,None,None,NaN,No,None,MODERNA,2
404847,1427475,87.0,F,Patient was hospitalized within 60 days of re...,Yes,05/27/2021,No,Yes,7.0,NaN,...,02/17/2021,0.0,NaN,NaN,NaN,NaN,No,NaN,MODERNA,2


In [24]:
# Extract relevant columns for analysis 
covid_adv = pat.loc[:, ['VAERS_ID', 'AGE_YRS', 'SEX', 'DIED', 'CUR_ILL', 'HISTORY', 'DISABLE', 'BIRTH_DEFECT', 'L_THREAT', 'HOSPITAL', 'VAX_MANU' ]]

In [ ]:
covid_adv.to_csv('covid_adv_pat.csv', index = False)